In [1]:
import requests 
import inquirer
import pandas as pd
import numpy as np
from config import riot_api

In [2]:
# region code is [{BR1:"Brazil",EUN1:"EU Nordic & East",EUW1:"EU West",JP1:"Japan",KR:"Korea",LA1:"Latin America North" 
# LA2:"Latin America South,NA1:"North America,OC1:"Oceania",PBE1:"Public Beta Environment,RU:"Russia",TR1:Turkey}]

region_code = input("Enter Your Region: ").upper() # Look up by Region

if region_code in ["NA1", "BR1", "LA1", "LA2", "OC1"]:
    region = "americas"
elif region_code in ["KR", "JP1"]:
    region = "asia"
else:
    region = "europe"
    

# Free Champions Rotation

url = f"https://{region}.api.riotgames.com/lol/platform/v3/champion-rotations?api_key=" + riot_api
champ_list = requests.get(url).json()

# Live Games

url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/featured-games?api_key=" + riot_api
live_games = requests.get(url)
statcode = live_games.status_code
if statcode == 200:
    live_games_status = "Region availible"
else:
    live_games_status ="Region unavailible"

live_games_status
print(f"{live_games_status}\n{live_games.status_code}\n{champ_list}") 


Enter Your Region: na1
Region availible
200
{'status': {'message': 'Bad request - Invalid routing value americas', 'status_code': 400}}


In [3]:
live_games = live_games.json()
live_games_df = pd.DataFrame(live_games['gameList'])

live_games_df

,gameId,mapId,gameMode,gameType,gameQueueConfigId,participants,observers,platformId,bannedChampions,gameStartTime,gameLength
0,4293902011,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 6, 'spell2Id': 4,...",{'encryptionKey': '9pWQUS5CwgkGL36UF5f/b7q8OZ7...,NA1,[],1651376451644,170
1,4293902567,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 1,...",{'encryptionKey': 'u8hg8Ip+QGN3pt4evaTE7iJD6HY...,NA1,[],1651376608706,13
2,4293882624,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 3, 'spell2Id': 13...",{'encryptionKey': '7NeL3sUiduKqHH3jJndw+koxqYB...,NA1,[],1651376545463,76
3,4293872471,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 32...",{'encryptionKey': 'O9ig4ikvdGqL1yD4h9DnF5kN76g...,NA1,[],1651376321919,300
4,4293872048,12,ARAM,MATCHED_GAME,450,"[{'teamId': 100, 'spell1Id': 4, 'spell2Id': 7,...",{'encryptionKey': 'hksxkxfdu7iY4B95GTdoi7SprkD...,NA1,[],1651376334553,287


In [4]:
name = input("Enter Your Summoner Name: ") # And Username

Enter Your Summoner Name: cookiemonster123


In [5]:
# Summoners Data 

def Summoner (region_code, name):    
    
    url = f"https://{region_code}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=" + riot_api
    summoner_info_1 = requests.get(url).json()
    iden, puuid = summoner_info_1["id"], summoner_info_1["puuid"]
    
    Ranked_info_url = f"https://{region_code}.api.riotgames.com/lol/league/v4/entries/by-summoner/{iden}?api_key=" + riot_api
    Champ_Pool_Mastery_url = f"https://{region_code}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{iden}?api_key=" + riot_api

    Ranked_info = requests.get(Ranked_info_url).json()    
    Champ_Pool_Mastery = requests.get(Champ_Pool_Mastery_url).json()
    if live_games_status == "Region availible":
        url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{iden}?api_key=" + riot_api
        in_game_info =  requests.get(url)
        if in_game_info.status_code == "200":
            in_game_info =  in_game_info.json()
            print("In Live Game")
        else:
            in_game_info = np.nan
            print("No Live Data")
    else:
        in_game_info = np.nan
        
    return [summoner_info_1, Ranked_info, Champ_Pool_Mastery, in_game_info]

summoner_info = Summoner(region_code, name)
Summoner_info, Ranked_info, Champ_Pool_Mastery = [summoner_info[0]], summoner_info[1], summoner_info[2]



No Live Data


In [6]:
Summoner_info_df = pd.DataFrame(Summoner_info)

Summoner_info_df.head()

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...,M7O-YmNT9NAlWIvhaiYqqEB76TMuslBGbDvUr4ufmZVkB9...,2qAtVUWwnDmzrh3fy2QZXLUFE3QKMveBF1JpZVtqA0tfD8...,COOKIEMONSTER123,907,1651281208000,108


In [7]:
if not Ranked_info:
    print("Unranked this Season")
    Ranked_info_df = pd.DataFrame()

else:
    Ranked_info_df = pd.DataFrame(Ranked_info)

Ranked_info_df.head()

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,7a7ed076-d062-3079-ae71-e301c671c869,RANKED_SOLO_5x5,MASTER,I,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...,COOKIEMONSTER123,216,518,498,False,False,False,False


In [8]:
print(f"{region_code}, {region}, {name}")
Champ_Pool_Mastery_df = pd.DataFrame(Champ_Pool_Mastery)
Champ_Pool_Mastery_df.head()

NA1, americas, cookiemonster123


,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,chestGranted,tokensEarned,summonerId
0,111,5,131126,1649885306000,109526,0,True,2,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...
1,555,5,93483,1651194296000,71883,0,True,2,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...
2,43,5,92617,1648494144000,71017,0,True,2,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...
3,497,5,36765,1647523028000,15165,0,True,2,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...
4,53,5,23932,1648851907000,2332,0,True,1,YqQJT1EHJT8zHDIcgznkbcjs-MyZ58knXD0KnNqYDQ6Yu5...


In [9]:
def Games(puuid, region):
    
    last_20_id_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key=" + riot_api
    last_20_id = requests.get(last_20_id_url).json()
    
    match_data = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}?api_key=" + riot_api).json()) 
                       for match in last_20_id] 
    time_line = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}/timeline?api_key=" + riot_api).json()) 
                       for match in last_20_id]
    
    return [last_20_id, match_data, time_line]

puuid = Summoner_info_df["puuid"][0]
Last_20_Games_Metadata = Games(puuid, region)
Last_20_Games_Metadata_df = pd.DataFrame(Last_20_Games_Metadata)

Last_20_Games_Metadata_df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NA1_4292752414,NA1_4292677676,NA1_4292682830,NA1_4292649927,NA1_4292424896,NA1_4292460058,NA1_4292251124,NA1_4291644631,NA1_4291548880,NA1_4291583341,NA1_4291508995,NA1_4291523329,NA1_4291428930,NA1_4291434783,NA1_4291431826,NA1_4291397656,NA1_4290952848,NA1_4289961303,NA1_4289885962,NA1_4289813438
1,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."
2,"{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '...","{'metadata': {'dataVersion': '2', 'matchId': '..."


In [10]:
# Ranked Data

queue = input("Enter Your Queue Type: ") # Enter Queue Type [RANKED_SOLO_5x5, RANKED_TFT,
                                        # RANKED_FLEX_SR, RANKED_FLEX_TT]
tier = input("Enter Your Tier: ") # Enter Tier Level [IRON, BRONZE, SILVER, GOLD, 
                                        # PLATINUM, DIAMOND, MASTER, GRANDMASTER, CHALLENGER]
division = input("Enter Your Division: ") # Enter Division Level [IV, III, II, I]
page = 1 #input("Page Number: ")
url = f"https://{region_code}.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page}&api_key=" + riot_api
rank_info = requests.get(url).json()
rank_info_df = pd.DataFrame(rank_info)
rank_info_df


Enter Your Queue Type: RANKED_SOLO_5x5
Enter Your Tier: CHALLENGER
Enter Your Division: I


,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,xqgupnGKgEbWdIQx_y3DdErBnIxHP1HOBf_ztK301B-HFy8,from mcdonald,1766,177,116,True,False,False,True
1,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,30o76MiQlVXN5xgY35KM1SBNk-jJhxGLqyQi4CHIOrwib5s,Sheiden,1716,171,106,True,False,False,False
2,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,8WvejROpZIS6Y1GkmWSx29RLlFfw4WyYYtVzddb1KI9ubA...,TwtchTV Sheiden,1619,187,139,True,False,False,False
3,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,heCKRUFKXoYSVMw0EbqH0slUTp33iam0_V5DhT8jMclWPt...,I will trade,1596,165,118,True,False,False,False
4,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,_RZZmTcCUibuSvTq4Awyldq1bkibIal2GAGy6sh02VjgS2...,optimabiee1,1561,188,158,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,2YSUfhGb0H4029-YxDPHDllA4tc-CvY4edIxUKdTLNqAmi...,skétch,836,73,60,False,False,False,False
201,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,B-P50KG72YqF42kYCTkbbRtQyBrsivb3-GlzDxWRqN_Fs7mm,20031028,835,138,104,False,False,False,False
202,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,72Doa9MZcxzYf7046EwOxFEq4Cw6SVcfY09tCIs3Ehsgyx4,Thats Not Fervor,835,134,116,False,False,True,True
203,d609d862-7a3f-3f35-8c18-1fbc8198f70a,RANKED_SOLO_5x5,CHALLENGER,I,fMv-hAiV-XzxjSNmehMYB9Fk3HeCDIDCHk_cJ6NNaj8pgUs,Nader,833,117,87,False,False,False,True
